# Notebook Summary
This note book go through the agent example provided in langchain. \
https://python.langchain.com/docs/modules/agents/quick_start \
https://python.langchain.com/docs/modules/agents/how_to/custom_agent

In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools.retriever import create_retriever_tool
import langchain_util as util
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_openai_functions_agent, create_openai_tools_agent
from langchain.agents import AgentExecutor
from langchain.agents import tool
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser


In [5]:
# Load the .env file which contains the API keys
load_dotenv()

# Set the API keys as environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_TRACING_V2']= 'true'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ["TAVILY_API_KEY"] = os.getenv('TAVILY_API_KEY')

# Optional, add tracing in LangSmith.
# This will help you visualize and debug the control flow
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "Agentic_RAG_LANGGRAPH"

## Create the agemnt

## Agent with tools that we provided as function
We define the tool that the agent can use as functions. \
We treat the tool just as we use a decorator in python

In [6]:
# This cell contains the definition of the tools that will be used in the agent.
# The tools are functions that can be called by the agent to perform specific tasks.

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def get_my_name() -> str:
    """Returns the name of the user."""
    return "John"

In [7]:
# Define the llm.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Summarize the tools for the agent.
tools = [get_word_length, get_my_name]

# create prompt for the agent.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Define the llm with the tools we build.
llm_with_tools = llm.bind_tools(tools)

# Define the agent. Where the input is the user input and the agent_scratchpad is the intermediate steps.
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

# Create the agent executor.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run the agent.
list(agent_executor.stream({"input": "What is my name?"}))


## The agent with added memory
This just means we added a variable of ***chat_history*** \
so the llm can remeber what haven been discussed and more mimic the real world case\
The ***chat_history*** is a dict type, where we need keep update it throughout the conversation


In [14]:
# Define the chat history key and add it to the prompt.
MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at current tasks.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY), # Here we insert the history.
        ("user", "{input}"), # Here is where we start the conversation.
        MessagesPlaceholder(variable_name="agent_scratchpad"), # This is where agent has acess to the tools.
    ]
)

chat_history = []

# A simple agent then the one above. Only difference is that we have a chat history.
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})

# Add the user input and the agent response to the chat history.
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" has 5 letters.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5Yes, "educa" is a real word with 5 letters.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='The word "educa" has 5 letters.')],
 'output': 'Yes, "educa" is a real word with 5 letters.'}

## Streaming Agent
In my opinion, the stream agent have the ability to stream the internal step going on while executing. 
More importantrly. The agent has ability to think the stpes to take to reach the final answer.\

In [5]:
# define tools 
import random

# @tool
# async def where_cat_is_hiding() -> str:
#     """Where is the cat hiding right now?"""
#     return random.choice(["under the bed", "on the shelf"])
@tool
def where_cat_is_hiding() -> str: # Feels like this doesn't need to be async
    """Where is the cat hiding right now?"""
    return random.choice(["under the bed", "on the shelf"])

# @tool
# async def get_items(place: str) -> str:
#     """Use this tool to look up which items are in the given place."""
#     if "bed" in place:  # For under the bed
#         return "socks, shoes and dust bunnies"
#     if "shelf" in place:  # For 'shelf'
#         return "books, penciles and pictures"
#     else:  # if the agent decides to ask about a different place
#         return "cat snacks"

@tool
def get_items(place: str) -> str:
    """Use this tool to look up which items are in the given place."""
    if "bed" in place:  # For under the bed
        return "socks, shoes and dust bunnies"
    if "shelf" in place:  # For 'shelf'
        return "books, penciles and pictures"
    else:  # if the agent decides to ask about a different place
        return "cat snacks"

In [6]:
# Get the prompt to use - you can modify this! 
# print(prompt.messages) -- to see the prompt
prompt = hub.pull("hwchase17/openai-tools-agent")

# Define model
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Define the tools
tools = [get_items, where_cat_is_hiding]

agent = create_openai_tools_agent(
    model.with_config({"tags": ["agent_llm"]}), tools, prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools).with_config(
    {"run_name": "Agent"}
)

In [10]:
# Note: We use `pprint` to print only to depth 1, it makes it easier to see the output from a high level, before digging in.
import pprint

chunks = []

async for chunk in agent_executor.astream(
    # This input is a bit more interesting, as it asks about the items in the place where the cat is hiding.
    # it is a bit more complex, as it requires the agent to first ask where the cat is hiding, and then ask about the items in that place.
    {"input": "what's items are located where the cat is hiding?"}
):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)

------
{'actions': [...], 'messages': [...]}
------
{'messages': [...], 'steps': [...]}
------
{'actions': [...], 'messages': [...]}
------
{'messages': [...], 'steps': [...]}
------
{'messages': [...],
 'output': 'The items located where the cat is hiding (on the shelf) are '
           'books, pencils, and pictures.'}


In [15]:
# This cell demonstrates how to use the agent executor the query 
# We added the print so we can see how the agent 'thinking' and what it is doing at each step.

async for chunk in agent_executor.astream(
    {"input": "what's items are located where the cat is hiding?"}
):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")

Calling Tool: `where_cat_is_hiding` with input `{}`
---
Tool Result: `on the shelf`
---
Calling Tool: `get_items` with input `{'place': 'on the shelf'}`
---
Tool Result: `books, penciles and pictures`
---
Final Output: The items located where the cat is hiding (on the shelf) are books, pencils, and pictures.
---


In [16]:
async for event in agent_executor.astream_events(
    {"input": "where is the cat hiding? what items are in that location?"},
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: Agent with input: {'input': 'where is the cat hiding? what items are in that location?'}
--
Starting tool: where_cat_is_hiding with inputs: {}
Done tool: where_cat_is_hiding
Tool output was: on the shelf
--
--
Starting tool: get_items with inputs: {'place': 'on the shelf'}
Done tool: get_items
Tool output was: books, penciles and pictures
--
The| cat| is| hiding| on| the| shelf|.| In| that| location|,| you| can| find| books|,| pencils|,| and| pictures|.|
--
Done agent: Agent with output: The cat is hiding on the shelf. In that location, you can find books, pencils, and pictures.


In [1]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:5]

C:\Users\yyb98\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[-0.00318459689536844,
 0.0110777294721545,
 -0.0041049622618212454,
 -0.011744660768894723,
 -0.000993727627486321]